# Lab01: Web Crawler.

- MSSV: 19120689
- Họ và tên: Lại Khánh Toàn

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file `Lab01-WebCrawler.ipynb`, `links.txt`, `images.txt` vào, rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

**Nội dung bài tập**

Sử dụng module regex và request để thu thập đường dẫn và hình ảnh từ một trang web.

## Part 1: Thu thập dữ liệu đơn giản

### 1.1. Import

In [1]:
import requests
import re

### 1.2. Thu thập liên kết

In [2]:
r = requests.get("http://www.hcmus.edu.vn")

# TODO:
# Displays all the links present in the specified root URL - "http://www.hcmus.edu.vn"
# use regex to find all urls from r.text (http[s]://...)

url_pattern = "http[s]?://(?:[-\w.])+"
urls = re.findall(url_pattern, r.text)


# Checking results?
for i in urls:
    print(i)

#Save urls to links.txt
# fp = open('links.txt', 'w')
# for i in urls:
#     fp.write(i)
#     fp.write('\n')
# fp.close()

http://www.w3.org
http://ogp.me
http://ogp.me
http://ogp.me
https://www.hcmus.edu.vn
https://www.hcmus.edu.vn
https://www.hcmus.edu.vn
https://www.hcmus.edu.vn
https://www.hcmus.edu.vn
http://www.hcmus.edu.vn
http://portal.hcmus.edu.vn
http://mail.office365.com
https://www.google.com
http://mail.google.com
http://www.hcmus.edu.vn
http://hocphi.hcmus.edu.vn
https://www.facebook.com
https://youtu.be
https://drive.google.com
https://www.hcmus.edu.vn
http://doantn.hcmus.edu.vn
http://congdoan.hcmus.edu.vn
https://www.hcmus.edu.vn
https://jobs.hcmus.edu.vn
https://drive.google.com
http://tuyensinh.hcmus.edu.vn
https://sdh.hcmus.edu.vn
https://drive.google.com
https://sdh.hcmus.edu.vn
http://www.e-learning.vn
https://pdt.hcmus.edu.vn
https://research.hcmus.edu.vn
https://hnkh.hcmus.edu.vn
http://stdjns.scienceandtechnology.com.vn
http://www.bmrat.org
http://cellstemcell.org
http://www.progressinbiology.org
http://ajhs.biomedpress.org
https://www.hcmus.edu.vn
https://hnkh.hcmus.edu.vn
https:/

### 1.3. Thu thập hình ảnh

In [ ]:
# TODO:
# Displays the source of all the images present in the root URL:
# img_src=?


# Checking results?
# for i in img_src:
#     print(i)

fp = open('images.txt', 'w')
for i in img_src:
    fp.write(i)
    fp.write('\n')
fp.close()

## Part 2. Thu thập và thể hiện dữ liệu web

### 2.1. Import library

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string

### 2.2. HTML Parser


Bộ phân tích cú pháp HTML (HTML Parser): nhận HTML code và trích xuất thông tin liên quan như tiêu đề của trang, đoạn văn trong trang, tiêu đề trong trang, liên kết, văn bản in đậm, v.v.

In [2]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)

    def handle_endtag(self, tag):
        print("End tag :", tag)

    def handle_data(self, data):
        print("Data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>')

Start tag: html
Start tag: head
Start tag: title
Data  : Test
End tag : title
End tag : head


#### Loại bỏ các HTML tag không cần thiết bằng cách thiết lập filter

In [3]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

### 2.3 Thu thập nội dung trang Web

#### Trong bài tập này ta thể hiện tài liệu (hay nội dung trang web) với cấu trúc đơn giản như sau: 
- Gọi $D$ là một tập tài liệu chứa *n* tài liệu: $D=\left\{d_1,d_2,...,d_n\right\}$.
- Ta thể hiện tài liệu bằng một dictionary `data={}` với `data[word]=[[url_idx_1,url_idx_2,...],frequency]` với `url_index`$\in{\left[{1,n}\right]}$

VD: `data['at']=[[1,2], 5]`
Từ `at` xuất hiện trong đường dẫn có index `1` và `2` tổng số lần xuất hiện là 5.

#### Bước 1: liệt kê các từ xuất hiện trong trang web:

In [4]:
def wordList(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []

    # TODO:
    # Với mỗi chuỗi trong filtered_text:
    #   Thay thế các dấu câu thành khoảng trắng (gợi ý: danh sách các dấu câu: string.punctuation; thay thế: .replace(...))
    #   Tách chuỗi bởi khoảng trắng (.split(...))
    #   Thêm các từ vừa được tách ra vào word_list
    for word in filtered_text:
        word = word.replace(string.punctuation, " ")
        word = word.split(" ")
        word_list.extend(word)
    return word_list
  
# Test
print(wordList('https://en.wikipedia.org/wiki/Web_mining'))

['Main', 'menu', 'Main', 'menu', 'move', 'to', 'sidebar', 'hide', 'Main', 'page', 'Contents', 'Current', 'events', 'Random', 'article', 'About', 'Wikipedia', 'Contact', 'us', 'Donate', 'Help', 'Learn', 'to', 'edit', 'Community', 'portal', 'Recent', 'changes', 'Upload', 'file', 'Languages', 'Language', 'links', 'are', 'at', 'the', 'top', 'of', 'the', 'page', 'across', 'from', 'the', 'title.', 'Search', 'Create', 'account', 'Log', 'in', 'Personal', 'tools', 'Create', 'account', 'Log', 'in', 'learn', 'more', 'Contributions', 'Talk', 'Contents', 'move', 'to', 'sidebar', 'hide', '(Top)', '1', 'Etymology', '2', 'Background', '3', 'Process', '3.1', 'Pre-processing', '3.2', 'Data', 'mining', '3.3', 'Results', 'validation', '4', 'Research', '5', 'Standards', '6', 'Notable', 'uses', '7', 'Privacy', 'concerns', 'and', 'ethics', '7.1', 'Situation', 'in', 'Europe', '7.2', 'Situation', 'in', 'the', 'United', 'States', '8', 'Copyright', 'law', '8.1', 'Situation', 'in', 'Europe', '8.2', 'Situation', '

#### Bước 2: Tính tần suất xuất hiện của từ trong 1 trang web, lưu trữ dữ liệu vào data:

In [ ]:
def read_url(url, url_idx, data):
    word_list = wordList(url)
    # TODO
    # Với mỗi từ w trong word_list:
    #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
    #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
    for word in word_list:
        if word not in data:
            data[word] = [[url_idx], 1]
        else:
            if url_idx not in data[word][0]:
                data[word][0].append(url_idx)
            data[word][1] += 1
    

# Test
test_data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, test_data)
test_data

#### Bước 3: Chạy chương trình lưu kết quả vào file output.txt

In [ ]:
data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, data)
read_url('https://en.wikipedia.org/wiki/Data_mining', 2, data)

sorted_keys = sorted(data.keys())

with open("output.txt", "w", encoding="utf-8") as f:
    output_line = "Word".ljust(20) + "Frequency".ljust(15) + "URL_idx".ljust(15) + "\n"
    f.writelines(output_line)
    f.writelines('---------------------------------------------------------------------\n\n')
    for key in sorted_keys:
        output_string = str(key).ljust(20) + str(data[key][1]).ljust(15) + str(data[key][0]).ljust(15) + "\n"
        f.writelines(output_string)


In [ ]:
# TODO
# Keyword tìm kiếm: python pickle
# Dùng pickle để lưu dictionary data xuống đĩa. Sau đó đọc lên và in ra 10 giá trị đầu tiên của nó.

# 1. Lưu data
import pickle
with open("pickle_file", "wb") as f:
    pickle.dump(f)

# 2. Đọc lên và in ra
with open("pickle_file", "rb") as f:
    file_data = pickle.load(f)

import itertools
result = dict(itertools.islice(file_data.items(), 10))
for key, val in result.items():
  print(f'\'{key}\': {val}')